In [5]:
import cv2
import numpy as np
import dlib

# DNNベースの顔検出器のロード
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# カメラの指定
cap = cv2.VideoCapture(0)

# 各パーツのリサイズ比を入力
eyex, eyey = 1.3, 1.5
mouthx, mouthy = 1.2, 1.2
nose_factor = 1.2  # 鼻全体を高くする倍率
small_nose_factor = 0.8  # 小鼻を小さくする倍率

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_detector(gray)
    
    for face in faces:
        # 顔のランドマークを取得
        landmarks = landmark_predictor(gray, face)

        # 左目の座標（ランドマークから取得）
        left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
        right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])

        # 左目の領域を拡大
        left_eye_rect = cv2.boundingRect(left_eye_points)
        x, y, w, h = left_eye_rect
        left_eye_roi = frame[y:y + h, x:x + w]
        big_left_eye = cv2.resize(left_eye_roi, None, fx=eyex, fy=eyey)
        
        # 新しいサイズが元の領域を越えないように調整
        new_height, new_width = big_left_eye.shape[:2]
        target_height = y + new_height if y + new_height < frame.shape[0] else frame.shape[0]
        target_width = x + new_width if x + new_width < frame.shape[1] else frame.shape[1]
        frame[y:target_height, x:target_width] = big_left_eye[:(target_height - y), :(target_width - x)]

        # 右目の領域を拡大
        right_eye_rect = cv2.boundingRect(right_eye_points)
        x, y, w, h = right_eye_rect
        right_eye_roi = frame[y:y + h, x:x + w]
        big_right_eye = cv2.resize(right_eye_roi, None, fx=eyex, fy=eyey)
        
        # 新しいサイズが元の領域を越えないように調整
        new_height, new_width = big_right_eye.shape[:2]
        target_height = y + new_height if y + new_height < frame.shape[0] else frame.shape[0]
        target_width = x + new_width if x + new_width < frame.shape[1] else frame.shape[1]
        frame[y:target_height, x:target_width] = big_right_eye[:(target_height - y), :(target_width - x)]

        # 口の領域を拡大
        mouth_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
        mouth_rect = cv2.boundingRect(mouth_points)
        x, y, w, h = mouth_rect
        mouth_roi = frame[y:y + h, x:x + w]
        big_mouth = cv2.resize(mouth_roi, None, fx=mouthx, fy=mouthy)
        
        # 新しいサイズが元の領域を越えないように調整
        new_height, new_width = big_mouth.shape[:2]
        target_height = y + new_height if y + new_height < frame.shape[0] else frame.shape[0]
        target_width = x + new_width if x + new_width < frame.shape[1] else frame.shape[1]
        frame[y:target_height, x:target_width] = big_mouth[:(target_height - y), :(target_width - x)]

        # 鼻を高くする（縦方向のスケーリング）
        nose_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(27, 36)])
        nose_rect = cv2.boundingRect(nose_points)
        x, y, w, h = nose_rect
        nose_roi = frame[y:y + h, x:x + w]
        big_nose = cv2.resize(nose_roi, None, fx=1.0, fy=nose_factor)  # 縦方向のみ拡大
        
        # 新しいサイズが元の領域を越えないように調整
        new_height, new_width = big_nose.shape[:2]
        target_height = y + new_height if y + new_height < frame.shape[0] else frame.shape[0]
        target_width = x + new_width if x + new_width < frame.shape[1] else frame.shape[1]
        frame[y:target_height, x:target_width] = big_nose[:(target_height - y), :(target_width - x)]

        # 小鼻を小さくする（左右の小鼻の部分を縮小）
        small_nose_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(31, 36)])
        small_nose_rect = cv2.boundingRect(small_nose_points)
        x, y, w, h = small_nose_rect
        small_nose_roi = frame[y:y + h, x:x + w]
        small_nose = cv2.resize(small_nose_roi, None, fx=small_nose_factor, fy=small_nose_factor)
        
        # 新しいサイズが元の領域を越えないように調整
        new_height, new_width = small_nose.shape[:2]
        target_height = y + new_height if y + new_height < frame.shape[0] else frame.shape[0]
        target_width = x + new_width if x + new_width < frame.shape[1] else frame.shape[1]
        frame[y:target_height, x:target_width] = small_nose[:(target_height - y), :(target_width - x)]

    # フレームを表示
    cv2.imshow('Real-time Face Modification', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

# カメラフレームサイズを指定
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 顔検出
    rects = detector(gray, 1)

    for rect in rects:
        # ランドマークを取得
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # ランドマーク68箇所のプロットと数字の表示
        for i, (x, y) in enumerate(shape):
            # 各ランドマークの位置に円を描画
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            # 各ランドマークの位置に番号を表示
            cv2.putText(image, str(i + 1), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)

    # 画像を表示
    cv2.imshow('Frame', image)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()
